In [2]:
import json
import re
import spacy
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) 

def lemmatize_text(text):
    doc = nlp(text)
    txt = [token.lemma_ for token in doc]
    return ' '.join(txt)
    
def extract_first_item(match):
    return match.group()[:5]

In [1]:
year_dict = {year: 0 for year in range(1994, 2025)}

In [13]:
# Initialize NLTK's WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

replace_pairs = [['\n',' '],['-',' '],[' \" a','oa'],['\" a','ae'],['\"a','ae'],[' \" o','oe'],['\" o','oe'],['\"o','oe'],[' \" u','ue'], ['\" u','ue'],['\"u','ue'],[' \' a','a'],[' \' e','e'],[' \' o','o'],["\' ", ""],["\'", ""],['  ',' '],['  ',' ']]

def replace_string(text):
    for old, new in replace_pairs:
        text = text.replace(old, new)
    return text

def preprocess_text(text):
    # Remove non-alphanumeric characters and lowercasing
    text = replace_string(text)
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    text = remove_stopwords(text)

    # Tokenize the text
    tokens = word_tokenize(text)
    # Lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Remove stopwords
    
    return lemmatized_tokens

# Runtime: ~2 minutes            
with open('saved_files/arxiv_preprocessed.csv','w', newline='') as file:
    for line in tqdm(open("saved_files/arxiv-metadata-oai-snapshot.json", 'r')): 
        temp = json.loads(line) 
        
        if temp["categories"] == "quant-ph":
            import pandas as pd 
            ds = temp["versions"][0]["created"]
            year = ds.split()[3]

            year_dict[int(year)] += 1
            if int(year) > 2023:
                continue 
            
             
            abstract = temp.get("abstract", "").strip()            
            processed_abstract = preprocess_text(abstract)

            if processed_abstract == []:
                continue

            id_str = str(temp["id"])
            
            save_txt = "{},{},{}".format(id_str, ' '.join(processed_abstract), str(temp["versions"][0]["created"]).replace(",","") )

            file.write(save_txt)
            file.write('\n') 

2426574it [00:42, 57383.95it/s] 
